# Router

# Code Overview

## Section 1: Initialization & Configuration
- Sets up a logger using `quill`, then initializes and loads configuration from a `ConfigManager` object.
- Applies configuration values such as log levels and packet handling flags.
- Prepares a raw socket for packet forwarding and allocates shared data structures.

## Section 2: Threaded Packet Handling
- Spawns multiple threads for different packet processing functions:
  - **thread_client_to_server**: Captures packets from a given interface and forwards them to another host.
  - **thread_server_to_client**: Captures packets from the server side and processes them internally.
  - **thread_serialization**: Periodically serializes collected packet data (flags, options, logs) into JSON files.

## Section 3: Data Serialization & Cleanup
- Uses a `serialization_manager` function to periodically dump counts of TCP flags, TCP options, and packet logs into JSON files.
- Upon startup, deletes old JSON logs and starts fresh.
- Maintains mutex-protected data structures for thread-safe logging and statistics collection.
- In an infinite loop until interrupted, and on exit, closes the raw socket, ensuring proper cleanup.

Below is a summarized discussion of the provided code, along with a conceptual diagram (in text form), and explanations of some of the more complex snippets.

---

## High-Level Discussion

This code is part of a packet processing pipeline that involves capturing packets from a network interface, processing and logging TCP information, and then reordering and sending these packets back out. The main functionalities are:

1. **Packet Capture and Filtering:**  
   Using `libpcap` (functions like `pcap_open_live`, `pcap_loop`), the code sets filters to capture packets from a specific host (`192.168.2.2`) while excluding packets originating from a particular MAC address.  
   
   Example filter:  
   ```c
   "src host 192.168.2.2 and not ether src <some:mac:address>"
   ```

   This ensures we only process packets from the desired source while ignoring potentially unwanted traffic.

2. **Threaded Operation and Shared Data:**  
   Multiple threads operate on shared data structures, like a `packet_queue`, `tcp_flag_counts`, `tcp_option_counts`, and a `packet_log`. Each of these data structures is protected by mutexes to ensure thread-safe access. A `std::shared_ptr<Server::Data>` is passed around to give all threads access to these shared resources.

3. **Packet Analysis and Modification:**
   When a packet is received:
   - It extracts and parses Ethernet, IP, and TCP headers.
   - It logs TCP flags and options (for later JSON serialization).
   - It may modify the packet headers, such as removing certain TCP options (e.g., SACK) and recalculating checksums.
   - It stores these packets in a queue for delayed and/or reordered forwarding.

4. **Reordering and Sending Packets:**
   After a certain time threshold (e.g., 1ms), the code sends all queued packets in reverse order. This simulates a scenario where packets might arrive out of order, potentially for testing network resilience or debugging.

5. **Serialization and Logging:**
   Flags, options, and packet metadata are logged and periodically flushed to JSON files by a separate `serialization_manager` thread.

---

## Conceptual Diagram (Text-Based)

```
   +------------------------------------------------------+
   |                  Initialization                      |
   |  - Setup Logging                                     |
   |  - Load Config                                       |
   |  - Setup Raw Socket                                  |
   +-----------------------------+------------------------+
                                 |capture_packets_from() |
                                 v                       |
                          +-------------+                |
                          |  PCAP Loop  |                |
                          |  (from)     |                |
                          +------+------+                |
                                 | packet_handler_from() |
                                 v                       |
             +-------------------------------------------+------------+
             | Parse Headers (Ethernet, IP, TCP)                     |
             | - Log flags, options                                  |
             | - Add Packet to Queue                                 |
             +---------------------------+----------------------------+
                                     |time diff > 1ms? send_queued_packets() 
                                     v
                             +---------------+
                             | Reverse & Send|
                             | All Packets   |
                             +-------+-------+
                                     |
                                     v
                            +-----------------------+
                            |       Raw Socket      |
                            | sendto(...)           |
                            +-----------------------+
```

**Explanation of Flow:**  
- Packets enter through `pcap_loop`.
- Each packet is handled by `packet_handler_from`.
- After parsing and logging, they're queued.
- Once the time difference from the last packet exceeds a threshold (1ms), the queue is reversed, and all packets are sent out via the raw socket.

---

## Key Complex Snippets and Their Explanations

### Snippet 1: Computing Time Differences and Sending Queued Packets

```c
// Compute time difference in microseconds
long time_diff_us = compute_time_difference(internal->prev_timestamp, curr_timestamp);

// If first packet, time_diff_us = 0
if (internal->prev_timestamp.tv_sec == 0 && internal->prev_timestamp.tv_usec == 0) {
    time_diff_us = 0;
}

// If more than 1ms passed, send queued packets
if (time_diff_us > 1000) {
    send_queued_packets(internal);
}
```

**What’s Happening:**  
This snippet retrieves timestamps from incoming packets, computes how much time has passed since the previous packet. When that delay exceeds 1ms, it triggers the sending of all queued packets. This simulates a timing-based batch sending mechanism.

**Why it’s Complex:**  
Timing and synchronization are crucial. The code ensures that packets are only sent after a small delay, potentially to mimic network jitter or reordering conditions.

---

### Snippet 2: Processing TCP Options and Removing SACK

```c
// Check for and remove SACK options in the TCP header
if (ip_header->protocol == IPPROTO_TCP) {
    // ... parsing options ...
    if (sack_option_found) {
        // Rebuild the packet without SACK options
        // Adjust TCP header length, recalculate checksums
        compute_tcp_checksum(ip_header, tcp_header);
        compute_ip_checksum(ip_header);
    }
}
```

**What’s Happening:**  
This snippet identifies if the SACK (Selective Acknowledgment) option is present in the TCP header. If found, it rebuilds the TCP options without the SACK entries. This involves careful memory manipulation, option parsing, and length adjustments. After modifying the packet, it must recompute the IP and TCP checksums to maintain packet integrity.

**Why it’s Complex:**  
Manually parsing and modifying TCP options is error-prone. The code must carefully handle variable-length options, ensure no malformed data, and properly update all lengths and checksums.

---

### Snippet 3: Thread-Safe Logging and Incrementing Counts

```c
{
    std::lock_guard<std::mutex> lock(internal->flags_mutex);
    for (const auto &flag_name : flag_names) {
        internal->tcp_flag_counts[flag_name]++;
    }
}
```

**What’s Happening:**  
This uses a `std::lock_guard` to ensure that increments to `tcp_flag_counts` happen without data races. Multiple threads may process packets simultaneously, so proper locking prevents corruption of shared data.

**Why it’s Complex:**  
Multithreaded concurrency is tricky. Without locks, you risk inconsistent reads/writes. This snippet shows a simple but crucial concurrency control mechanism.

---

### Snippet 4: Reversing the Packet Queue and Sending

```c
{
    std::lock_guard<std::mutex> lock(internal->queue_mutex);

    // Reverse the queue
    std::reverse(internal->packet_queue.begin(), internal->packet_queue.end());

    // Send each packet
    for (const auto &pkt : internal->packet_queue) {
        send_packet(pkt, internal->logger, internal);
    }

    // Clear queue
    internal->packet_queue.clear();
    internal->total_payload_length = 0;
}
```

**What’s Happening:**  
The code acquires a lock on the queue, reverses the order of packets, sends them, and then clears the queue. Reversing is a simple way to simulate out-of-order delivery.

**Why it’s Complex:**  
Ensuring that no other thread is manipulating the queue at the same time is essential. The lock guarantees that the reverse, send, and clear operations happen atomically. The logic is conceptually straightforward but critical for ensuring thread safety and deterministic behavior.
